In [1]:
import pandas as pd
import requests
from datetime import datetime

# using colab secrets to store our passwords and keys
from google.colab import userdata

# Making the API call and viewing the json

In [2]:
city = 'Berlin'
API_key = userdata.get('openWeatherApi')

# check out the docs for more info on making an api call https://openweathermap.org/forecast5
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

response = requests.get(url)
json = response.json()

json

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1710255600,
   'main': {'temp': 14.79,
    'feels_like': 13.73,
    'temp_min': 12.28,
    'temp_max': 14.79,
    'pressure': 997,
    'sea_level': 997,
    'grnd_level': 1008,
    'humidity': 54,
    'temp_kf': 2.51},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 3.23, 'deg': 290, 'gust': 4.4},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2024-03-12 15:00:00'},
  {'dt': 1710266400,
   'main': {'temp': 13.33,
    'feels_like': 12.26,
    'temp_min': 10.4,
    'temp_max': 13.33,
    'pressure': 1003,
    'sea_level': 1003,
    'grnd_level': 1009,
    'humidity': 59,
    'temp_kf': 2.93},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'clouds': {'all': 33},
   'wind': {'speed': 2.54, 'deg': 280, 'gust': 6.29},
   'visibility': 1000

# Exploring the json
Remember, if you want to know more about any of the information in the json, or you're unsure what a key is, check out the [open weather docs](https://openweathermap.org/forecast5).

First, let's look at the keys

In [ ]:
json.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [ ]:
json["cod"]

'200'

In [ ]:
json["message"]

0

In [ ]:
json["cnt"]

40

In [ ]:
json["list"]

Looks like the information we'll want is in `list`. Let's look at the first element of `list` to investigate further.

In [ ]:
json["list"][0]

{'dt': 1702641600,
 'main': {'temp': 3.12,
  'feels_like': -0.39,
  'temp_min': 3.12,
  'temp_max': 3.74,
  'pressure': 1023,
  'sea_level': 1023,
  'grnd_level': 1025,
  'humidity': 90,
  'temp_kf': -0.62},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'clouds': {'all': 55},
 'wind': {'speed': 3.91, 'deg': 238, 'gust': 9.57},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2023-12-15 12:00:00'}

In [ ]:
json["city"]

{'id': 2950159,
 'name': 'Berlin',
 'coord': {'lat': 52.5244, 'lon': 13.4105},
 'country': 'DE',
 'population': 1000000,
 'timezone': 3600,
 'sunrise': 1702624234,
 'sunset': 1702651928}

# Building a DataFrame

## What would we like from the json?
Let's begin by exploring the first 2 items of `list` and see what we have.

In [ ]:
json["list"][0]

{'dt': 1702641600,
 'main': {'temp': 3.12,
  'feels_like': -0.39,
  'temp_min': 3.12,
  'temp_max': 3.74,
  'pressure': 1023,
  'sea_level': 1023,
  'grnd_level': 1025,
  'humidity': 90,
  'temp_kf': -0.62},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'clouds': {'all': 55},
 'wind': {'speed': 3.91, 'deg': 238, 'gust': 9.57},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2023-12-15 12:00:00'}

In [ ]:
json["list"][1]

{'dt': 1702652400,
 'main': {'temp': 4.22,
  'feels_like': 0.9,
  'temp_min': 4.22,
  'temp_max': 4.93,
  'pressure': 1028,
  'sea_level': 1028,
  'grnd_level': 1026,
  'humidity': 91,
  'temp_kf': -0.71},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'clouds': {'all': 79},
 'wind': {'speed': 4, 'deg': 259, 'gust': 9.28},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'n'},
 'dt_txt': '2023-12-15 15:00:00'}

When deciding what is useful, remember the project goals.

The time the forecast is for will be useful.

In [ ]:
json["list"][0]["dt_txt"]

'2023-12-15 12:00:00'

The temperature will be useful. Scooters don't have heaters!

In [ ]:
json["list"][0]["main"]["temp"]

3.12

The forecast will be useful. Scooters don't have a roof!

In [ ]:
json["list"][0]["weather"][0]["main"]

'Clouds'

If it's been recently raining will be useful. Wet conditions will affect scooter rentals.

In [ ]:
json["list"][0]["rain"]["3h"]

KeyError: ignored

> If you get an error on 'rain', it's because it hasn't rained in the last 3 hours. But we know this key exists, because, like good Data Scientists, we read the [docs](https://openweathermap.org/forecast5)

Wind speed will be useful. It's likely to affect whether people rent scooters or not.

In [ ]:
json["list"][0]["wind"]["speed"]

3.91

## Bringing together our chosen items

In [3]:
weather_items = []

for item in json["list"]:
    weather_item = {
        "forecast_time": item.get("dt_txt", None),
        "temperature": item["main"].get("temp", None),
        "forecast": item["weather"][0].get("main", None),
        "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
        "wind_speed": item["wind"].get("speed", None),
    }

    weather_items.append(weather_item)

weather_df = pd.DataFrame(weather_items)

weather_df.head()

,forecast_time,temperature,forecast,rain_in_last_3h,wind_speed
0,2024-03-12 15:00:00,14.79,Clear,0.0,3.23
1,2024-03-12 18:00:00,13.33,Clouds,0.0,2.54
2,2024-03-12 21:00:00,11.00,Clouds,0.0,2.81
3,2024-03-13 00:00:00,7.92,Clouds,0.0,2.38
4,2024-03-13 03:00:00,6.15,Clouds,0.0,2.08


The `rain` key in the JSON data is not always present. In the absence of this key, it's assumed that there hasn't been any recent rainfall. Since this key isn't always available, not encountering it during our loop can cause an error. To address this, we'll employ two `.get()` methods to handle the absence of the `rain` key.

> **Remember:** The second parameter of `.get()` serves as a fallback value, which is inserted into our dictionary if the key we're looking for doesn't exist.

The first `.get()` method checks for the existence of the `rain` key. If present, it will use the value associated with that key. If the `rain` key is missing, an empty dictionary will be used, preventing the code from breaking. The second `.get()` method checks for the `3h` key. If present, it will use its value. Otherwise, a default value of `0` will be used.

# Creating a function for multiple cities

In [ ]:
import pandas as pd
from datetime import datetime
from pytz import timezone

def get_weather_data(cities):
  # As we will be working in the cloud, computers can be located anywhere
  # Let's correct the computer timezone to our local timezone
  berlin_timezone = timezone('Europe/Berlin')
  API_key = userdata.get('openWeatherApi')
  weather_items = []

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # Added the time retrieved so we know when the forecast was made
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in json["list"]:
        weather_item = {
            # Added the city name, so the information is clear when looking at multiple cities
            "city": city,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "forecast": item["weather"][0].get("main", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)

  return weather_df

In [ ]:
get_weather_data(["Berlin", "Hamburg", "Munich"])

,city,forecast_time,temperature,forecast,rain_in_last_3h,wind_speed,data_retrieved_at
0,Berlin,2023-12-15 12:00:00,3.12,Clouds,0.00,3.91,2023-12-15 11:08:14
1,Berlin,2023-12-15 15:00:00,4.22,Clouds,0.00,4.00,2023-12-15 11:08:14
2,Berlin,2023-12-15 18:00:00,4.21,Clouds,0.00,3.67,2023-12-15 11:08:14
3,Berlin,2023-12-15 21:00:00,3.71,Clouds,0.00,3.42,2023-12-15 11:08:14
4,Berlin,2023-12-16 00:00:00,3.59,Clouds,0.00,3.92,2023-12-15 11:08:14
...,...,...,...,...,...,...,...
115,Munich,2023-12-19 21:00:00,2.16,Clear,0.00,3.10,2023-12-15 11:08:15
116,Munich,2023-12-20 00:00:00,1.82,Clear,0.00,4.37,2023-12-15 11:08:15
117,Munich,2023-12-20 03:00:00,2.43,Clouds,0.00,5.84,2023-12-15 11:08:15
118,Munich,2023-12-20 06:00:00,3.36,Rain,0.66,6.59,2023-12-15 11:08:15


# Making our function work with our SQL cities data
This section delves into a more advanced aspect of the project, which may be beyond the point many of you got to. However, it serves as an illustrative example of how to structure your work effectively. It demonstrates how we can compartmentalise various aspects of our data processing pipeline into distinct functions, enhancing code readability and facilitating debugging. It's a good idea to start writing your pipeline in a similar manner ready for the cloud.
> This part will only work locally, not on colab. You must also create your weather table in SQL first.

In [ ]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime

def retreiving_and_sending_data():
  connection_string = connection()
  cities_df = get_cities_data(connection_string)
  weather_df = get_weather_data(cities_df)
  send_weather_data(weather_df, connection_string)
  return "Data has been updated"

def connection():
  schema = "gans"
  host = "127.0.0.1"
  user = "root"
  password = "YOUR_PASSWORD_HERE"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def get_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

def get_weather_data(cities_df):
  berlin_timezone = timezone('Europe/Berlin')
  API_key = 'YOUR_API_HERE'
  weather_items = []

  for city in cities_df["city"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # As we are now using the data from our relational database
    # the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["city"] == city, "city_id"].values[0]

    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in json["list"]:
        weather_item = {
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "forecast": item["weather"][0].get("main", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

  return weather_df

def send_weather_data(weather_df, connection_string):
  weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

In [4]:
city_ids = [1, 2, 3]
city_names = ["Berlin", "Frankfurt", "Munich"]

cities_df = pd.DataFrame({"city_id": city_ids,
                          "city_name": city_names})

cities_df

,city_id,city_name
0,1,Berlin
1,2,Frankfurt
2,3,Munich


In [9]:
cities_df.loc[cities_df["city_name"] == "Munich", "city_id"].values[0]

3

In [ ]:
retreiving_and_sending_data()

ModuleNotFoundError: ignored